In [2]:
# Python code to convert video to audio
import moviepy.editor as mp
import torch
import librosa
import numpy as np
import soundfile as sf
import wave
from scipy.io import wavfile
from IPython.display import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [3]:
# Insert Local Video File Path 
clip = mp.VideoFileClip(r"./obamaDeepfake.mp4")
  
# Insert Local Audio File Path
clip.audio.write_audiofile(r"./transcript.wav")

MoviePy - Writing audio in ./transcript.wav


MoviePy - Done.


In [4]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
file_name = 'transcript.wav'
framerate = 16000
input_audio, _ = librosa.load(file_name, sr=framerate)

# Use a loop to process the audio file in segments, if it's too large to fit in memory
segment_length = 30  # in seconds
num_segments = int(np.ceil(len(input_audio) / (segment_length * framerate)))
transcription = ''
start_time = 0
for i in range(num_segments):
    start = i * segment_length * framerate
    end = (i + 1) * segment_length * framerate
    input_segment = input_audio[start:end]
    input_values = tokenizer(input_segment, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    segment_transcription = tokenizer.batch_decode(predicted_ids)[0]
    transcription += segment_transcription

print(transcription)


WE'RE ENTERING AN ERA IN WHICH OUR ENEMIES CAN MAKE IT LOOK LIKE ANY ONE IS SAYING ANYTHING AT ANY POINT IN TIME EVEN IF THEY WOULD NEVER SAY THOSE THINGS SO FOR INSTANCE THEY COULD HAVE ME SAY THINGS LIKE I KNOW KILMONGER WAS RIGHT OR BEN CARSON IS IN THE SUNKIM PLACE OR ABOUT THIS SIMPLY PRESIDENT TRUMP IS A TOTAL AND COMPLETE DIPSHIT NOW YOU SEE I WOULD NEVER SAYESE THINGS AT LEAST NOT IN A PUBLIC ADDRESS BUT SOME ONE ELSE WOULD SOME ONE LIKE GORDAN PEAL THIS IS A DANGEROUS TIME MOVING FORWARD WE NEED TO BE MORE VIGILANT WITH WHAT WE TRUST FROM THE INTERNET THAT'S A TIME WHEN WE NEED TO RELY ON TRUSTED NEW SOURCES MAY SOUND BASIC BUT HOW WE MOVE FORWARD IN THE AGE OF INFORMATION ISGOING TO BE THE DIFFERENCE BETWEEN WHETHER WE SURVIVE OR WHETHER WE BECOME SOME KIND OF FUCKED UP DYSTOPIA THANK YOU STAY WOKE BITCHES


In [ ]:
# Split the transcription into words
words = transcription.split()

# Calculate the duration of the audio data
wav = wave.open(file_name, 'r')
frames = wav.getnframes()
rate = wav.getframerate()
duration = frames / float(rate)
print(duration)

# Find the time stamps of the phonemes 'b', 'm', and 'p'
for i, word in enumerate(words):
    if 'B' in word or 'M' in word or 'P' in word:
        word_duration = duration / len(words)
        time_stamp = i * word_duration
        minutes, seconds = divmod(time_stamp, 60)
        milliseconds = int((seconds % 1) * 1000)
        print(f"Phoneme 'b', 'm', or 'p' found in word '{word}' at {int(minutes)}:{int(seconds)}.{milliseconds}")

72.47
Phoneme 'b', 'm', or 'p' found in word 'ENEMIES' at 0:3.170
Phoneme 'b', 'm', or 'p' found in word 'MAKE' at 0:4.76
Phoneme 'b', 'm', or 'p' found in word 'POINT' at 0:9.58
Phoneme 'b', 'm', or 'p' found in word 'TIME' at 0:9.964
Phoneme 'b', 'm', or 'p' found in word 'ME' at 0:16.758
Phoneme 'b', 'm', or 'p' found in word 'KILMONGER' at 0:19.476
Phoneme 'b', 'm', or 'p' found in word 'BEN' at 0:21.288
Phoneme 'b', 'm', or 'p' found in word 'SUNKIM' at 0:23.552
Phoneme 'b', 'm', or 'p' found in word 'PLACE' at 0:24.5
Phoneme 'b', 'm', or 'p' found in word 'ABOUT' at 0:24.911
Phoneme 'b', 'm', or 'p' found in word 'SIMPLY' at 0:25.817
Phoneme 'b', 'm', or 'p' found in word 'PRESIDENT' at 0:26.270
Phoneme 'b', 'm', or 'p' found in word 'TRUMP' at 0:26.723
Phoneme 'b', 'm', or 'p' found in word 'COMPLETE' at 0:28.987
Phoneme 'b', 'm', or 'p' found in word 'DIPSHIT' at 0:29.440
Phoneme 'b', 'm', or 'p' found in word 'PUBLIC' at 0:35.782
Phoneme 'b', 'm', or 'p' found in word 'BUT' at

In [ ]:
import torch
import librosa
import numpy as np
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC

# Load the audio file and resample it to the required sample rate
audio_file = 'reuben.wav'
sr = 16000
audio, _ = librosa.load(audio_file, sr=sr)

# Split the audio file into segments of 5 seconds
segment_len = 60 * sr
segments = [audio[i:i+segment_len] for i in range(0, len(audio), segment_len)]

# Load the Wav2Vec2 tokenizer and model
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Define the phonemes of interest
target_phonemes = ['E', 'M', 'B', 'P']

# Initialize the output array for the phoneme times
phoneme_times = {ph: [] for ph in target_phonemes}

# Iterate over the audio segments and predict the phonemes
for i, segment in enumerate(segments):
    input_values = tokenizer(segment, return_tensors='pt').input_values
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    predicted_phonemes = tokenizer.batch_decode(predicted_ids)[0]
    print(predicted_phonemes)

    # Add the times of occurrence of the target phonemes to the output array
    for ph in target_phonemes:
        ph_indices = [j for j, p in enumerate(predicted_phonemes) if p == ph]
        ph_times = [(i*segment_len + j)*1.0/sr for j in ph_indices]
        phoneme_times[ph].extend(ph_times)

# Print the output phoneme times
print('Phoneme times:')
for ph in target_phonemes:
    for i in range(len(phoneme_times[ph])):
        num = phoneme_times[ph][i]
        phoneme_times[ph][i] = format((int(num) + ((num - int(num))*1000)), ".3f")
    print(f'{ph}: {phoneme_times[ph]}')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
c:\Python\Python38\lib\site-packages\transformers\models\wav2vec2\tokenization_wav2vec2.py:752: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


WE ARE ENTERING AN ERROR IN WHICH OUR ENEMIES CAN MAKE IT LOOK LIKE ANY ONE IS SAYING ANYTHING AT ANY POINT IN TIME EVEN IF THEY WOULD NEVER SAY THOSE THINGS NOW YOU SEE I WOULD NEVER SAY THESE THINGS AT LEAST NOT IN A PUBLIC ADDRESS BUT SOME ONE ELSE WOULD SOME ONE LIKE JEORDON PM THIS IS A DANGEROUS TYIME MOVING FORWARD WE NEED TO BE MORE VIGILANT WITH WHAT WE TRUST FROM THE INTERNATE AT A TIME WHEN WE NEED TO RELY ON TRUSTED NEW SOURCES THIS MAY SOUND BASIC BUT HOW WE MOVE FORWARD IN THE AGE OF INFORMATION IS GOING TO BECOME THE DIFFERENCE BETWEEN WHETHER WE SURVIVE OR WHETHER WE BECOME SOME KIND OF DISTOPIA THANK YOU
Phoneme times:
E: ['0.062', '0.312', '0.438', '0.625', '1.188', '2.375', '2.500', '2.688', '3.312', '4.125', '4.625', '7.125', '7.250', '7.375', '7.875', '8.500', '8.625', '9.312', '10.438', '10.500', '11.188', '11.312', '11.875', '12.000', '12.812', '14.375', '15.062', '15.312', '15.438', '15.625', '16.312', '16.562', '16.875', '17.062', '18.562', '19.188', '20.312', 